In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
# Add massimal tools folder to path
import sys
sys.path.append("/massimal/python/tools")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pathlib

# Local imports
import annotation, hyspec_cnn

In [4]:
# Disable GPUs (in case of Tensorflow trying to use GPUs and raising errors)
#tf.config.set_visible_devices([], 'GPU')

# Enable eager mode
#tf.config.run_functions_eagerly(True)

In [5]:
# Check if GPU is used
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Parameters
OUTPUT_CHANNELS = 6
BATCH_SIZE = 4

In [7]:
# Paths 
# base_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS')
base_dir = pathlib.Path('/massimal/data/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS')

train_dataset_path = base_dir / '5b_Rad_Georef_SGC_PCA_TrainValSplit/Training_Tiles/Dataset'
val_dataset_path = base_dir / '5b_Rad_Georef_SGC_PCA_TrainValSplit/Validation_Tiles/Dataset'
unet_model_save_dir = base_dir /  'X_SavedKerasModels/InpaintedDataset_TestImages-12-17-24'
tensorboard_log_dir = base_dir / 'X_TensorboardLogs/InpaintedDataset_TestImages-12-17-24'

In [8]:
# Load datasets 
train_dataset = tf.data.experimental.load(str(train_dataset_path))
val_dataset = tf.data.experimental.load(str(val_dataset_path))

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


In [9]:
# Get number of tiles in each dataset, and dataset shape
n_tiles_train = train_dataset.cardinality()
n_tiles_val = val_dataset.cardinality()
tile_nrows,tile_ncols,tile_nchannels = train_dataset.element_spec[0].shape.as_list()
print(f'Number of training tiles: {n_tiles_train}')
print(f'Number of validation tiles: {n_tiles_val}')
print(f'Tile data shape (PCA tiles): {(tile_nrows,tile_ncols,tile_nchannels)}')


Number of training tiles: 457
Number of validation tiles: 107
Tile data shape (PCA tiles): (128, 128, 8)


In [10]:
# From https://www.tensorflow.org/tutorials/images/segmentation#optional_imbalanced_classes_and_class_weights
def add_sample_weights(image, label, name):
    # The weights for each class, with the constraint that:
    #     sum(class_weights) == 1.0
    class_weights = tf.constant([0.0, 1.0, 1.0, 1.0, 1.0, 1.0]) # Hard-coded for current dataset, zero weight for background
    class_weights = class_weights/tf.reduce_sum(class_weights)

    # Create an image of `sample_weights` by using the label at each pixel as an 
    # index into the `class weights` .
    sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

    return image, label, sample_weights

In [11]:
# Shuffle training dataset (tiles are originally ordered by image) and add sample weights
train_dataset = train_dataset.shuffle(buffer_size=n_tiles_train)
train_dataset = train_dataset.map(add_sample_weights)
val_dataset = val_dataset.map(add_sample_weights)

In [12]:
# Batch datasets
train_dataset_batch = train_dataset.batch(BATCH_SIZE)
val_dataset_batch = val_dataset.batch(BATCH_SIZE)

In [13]:
# for pca_tile, label_tile, weight_tile in val_dataset.take(3):
#     plt.imshow(label_tile)
#     plt.show()
#     plt.imshow(weight_tile)
#     plt.show()
    

In [14]:
# Create the U-Net model
unet = hyspec_cnn.unet(input_channels=tile_nchannels,
                       output_channels=OUTPUT_CHANNELS,
                       first_layer_channels=32,
                       depth = 1,
               )
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, None, None,  0           []                               
                                 8)]                                                              
                                                                                                  
 augmentation (Sequential)      (None, None, None,   0           ['input_image[0][0]']            
                                8)                                                                
                                                                                                  
 initial_convolution (Conv2D)   (None, None, None,   2336        ['augmentation[0][0]']           
                                32)                                                           

In [15]:
# Print layers with sublayers
for layer in unet.layers:
    print('----')
    print(layer.name)
    if hasattr(layer,'layers'):
        for l in layer.layers:
            print('\t'+l.name)
       # print(layer.layers)

----
input_image
----
augmentation
	random_flip
----
initial_convolution
----
downsamp_res_1/2
	conv2d
	batch_normalization
	leaky_re_lu
----
upsamp_res_1/1
	conv2d_transpose
	batch_normalization_1
	re_lu
----
skipconnection_res_1/1
----
classification


In [16]:
# Define callbacks
model_save_filename = str(unet_model_save_dir) + '/unet_model.epoch{epoch:02d}-loss{val_loss:.6f}-acc{val_sparse_categorical_accuracy:.3f}.hdf5'
callbacks =[tf.keras.callbacks.ModelCheckpoint(filepath = model_save_filename,
                                               save_best_only=True,
                                               verbose = 1),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, verbose=1),
#             tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.2, verbose=1),
            tf.keras.callbacks.TensorBoard(log_dir= tensorboard_log_dir)]

In [17]:
unet.compile(optimizer=tf.keras.optimizers.RMSprop(0.0001), 
             loss="sparse_categorical_crossentropy",
             weighted_metrics=['sparse_categorical_accuracy'], # Need weights to ignore background
             metrics = []) # Sparse because classes are numbered, not one-hot

In [ ]:
# Fit model to dataset
history = unet.fit(train_dataset.batch(BATCH_SIZE),
                   epochs=200,
                   validation_data=val_dataset.batch(BATCH_SIZE),
                   callbacks=callbacks)

Epoch 1/200
115/115 [==============================] - ETA: 0s - loss: 0.1135 - sparse_categorical_accuracy: 0.5359
Epoch 1: val_loss improved from inf to 0.10412, saving model to /massimal/data/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/X_SavedKerasModels/InpaintedDataset_TestImages-12-17-24/unet_model.epoch01-loss0.104118-acc0.423.hdf5
115/115 [==============================] - 7s 38ms/step - loss: 0.1135 - sparse_categorical_accuracy: 0.5359 - val_loss: 0.1041 - val_sparse_categorical_accuracy: 0.4233 - lr: 1.0000e-04
Epoch 2/200
113/115 [============================>.] - ETA: 0s - loss: 0.0841 - sparse_categorical_accuracy: 0.6468
Epoch 2: val_loss improved from 0.10412 to 0.09004, saving model to /massimal/data/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/X_SavedKerasModels/InpaintedDataset_TestImages-12-17-24/unet_model.epoch02-loss0.090037-acc0.499.hdf5
115/115 [==============================] - 4s 33ms/step - loss: 0.0843 - sparse_categorical_accuracy: 0.6443 - val_lo

In [ ]:
epochs = range(1, history.params['epochs'] + 1)
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
epochs = range(1, history.params['epochs'] + 1)
train_acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

plt.figure()
plt.plot(epochs, train_acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.show()